In [1]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import math
import seaborn as sns
import time


from kan_convolutional.KANLinear import KANLinear
from kan_convolutional.KANConv import KAN_Convolutional_Layer
from kan_convolutional import convolution 

import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class Data_Class(Dataset):
    def __init__(self, root_dirs, transform=None):
        self.transform = transform
        self.image_files = []
        self.labels = []
        self.class_names = []

        for root_dir in root_dirs:
            for label, subfolder in enumerate(os.listdir(root_dir)):
                subfolder_path = os.path.join(root_dir, subfolder)
                if os.path.isdir(subfolder_path):
                    if subfolder not in self.class_names:
                        self.class_names.append(subfolder)
                    label = self.class_names.index(subfolder)
                    for img_file in os.listdir(subfolder_path):
                        if img_file.endswith('.png'):
                            self.image_files.append(os.path.join(subfolder_path, img_file))
                            self.labels.append(label)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        image = Image.open(img_name).convert('L')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

    def get_class_names(self):
        return self.class_names

root_dirs = [
    "aug"
]

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = Data_Class(root_dirs=root_dirs, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

import torch
import torch.nn as nn
import torch.nn.functional as F

class SuperConvolutionalKAN(nn.Module):
    def __init__(self):
        super(SuperConvolutionalKAN, self).__init__()

        self.conv1 = nn.Conv2d(1, 5, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(5, 25, kernel_size=3, padding=1)

        self.maxpool = nn.MaxPool2d(kernel_size=(2, 2))

        self.flatten = nn.Flatten()

        self.kan1 = KANLinear(
            in_features=6400,
            out_features=256,
            grid_size=10,
            spline_order=3,
            scale_noise=0.01,
            scale_base=1,
            scale_spline=1,
            base_activation=nn.SiLU,
            grid_eps=0.02,
            grid_range=[0, 1]
        )

        self.kan2 = KANLinear(
            in_features=256,
            out_features=9, # out_features will change as per the number of output class of the dataset being considered. 
            grid_size=10,
            spline_order=3,
            scale_noise=0.01,
            scale_base=1,
            scale_spline=1,
            base_activation=nn.SiLU,
            grid_eps=0.02,
            grid_range=[0, 1]
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        
        x = self.conv2(x)
        x = self.maxpool(x)
        
        x = self.flatten(x)

        x = self.kan1(x)

        x = self.kan2(x)
        
        x = F.log_softmax(x, dim=1)

        return x
    
model = SuperConvolutionalKAN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

start_time = time.time()

for epoch in range(10): 
    epoch_start_time = time.time()
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_time = time.time() - epoch_start_time
    print(f'Epoch [{epoch + 1}/10], Loss: {running_loss / len(train_loader):.4f}, Time elapsed: {epoch_time:.2f} seconds')

total_time = time.time() - start_time
print(f"Training completed in: {total_time:.2f} seconds")

model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')

print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')
torch.save(model.state_dict(), 'super_kan_c_64x64.pth')

Epoch [1/10], Loss: 1.1303, Time elapsed: 220.53 seconds
Epoch [2/10], Loss: 0.5148, Time elapsed: 187.59 seconds
Epoch [3/10], Loss: 0.2709, Time elapsed: 176.61 seconds
Epoch [4/10], Loss: 0.1383, Time elapsed: 180.12 seconds
Epoch [5/10], Loss: 0.0947, Time elapsed: 160.86 seconds
Epoch [6/10], Loss: 0.1101, Time elapsed: 176.55 seconds
Epoch [7/10], Loss: 0.0485, Time elapsed: 162.82 seconds
Epoch [8/10], Loss: 0.0393, Time elapsed: 181.02 seconds
Epoch [9/10], Loss: 0.0324, Time elapsed: 174.93 seconds
Epoch [10/10], Loss: 0.0289, Time elapsed: 168.30 seconds
Training completed in: 1789.33 seconds
Accuracy: 0.9903, Precision: 0.9914, Recall: 0.9909, F1 Score: 0.9911


In [2]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import math
import seaborn as sns
import time


from kan_convolutional.KANLinear import KANLinear
from kan_convolutional.KANConv import KAN_Convolutional_Layer
from kan_convolutional import convolution 


import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class Data_Class(Dataset):
    def __init__(self, root_dirs, transform=None):
        self.transform = transform
        self.image_files = []
        self.labels = []
        self.class_names = []

        for root_dir in root_dirs:
            for label, subfolder in enumerate(os.listdir(root_dir)):
                subfolder_path = os.path.join(root_dir, subfolder)
                if os.path.isdir(subfolder_path):
                    if subfolder not in self.class_names:
                        self.class_names.append(subfolder)
                    label = self.class_names.index(subfolder)
                    for img_file in os.listdir(subfolder_path):
                        if img_file.endswith('.png'):
                            self.image_files.append(os.path.join(subfolder_path, img_file))
                            self.labels.append(label)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        image = Image.open(img_name).convert('L')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

    def get_class_names(self):
        return self.class_names

root_dirs = [
    "aug"
]

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = Data_Class(root_dirs=root_dirs, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

import torch
import torch.nn as nn
import torch.nn.functional as F

class NormalizedConvolutionalKAN(nn.Module):
    def __init__(self):
        super(NormalizedConvolutionalKAN, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 5, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(5)
        
        self.conv2 = nn.Conv2d(5, 25, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(25)

        self.maxpool = nn.MaxPool2d(kernel_size=(2, 2))

        self.flatten = nn.Flatten()

        self.kan1 = KANLinear(
            in_features=25 * 16 * 16,
            out_features=9, # out_features will change as per the number of output class of the dataset being considered. 
            grid_size=10,
            spline_order=3,
            scale_noise=0.01,
            scale_base=1,
            scale_spline=1,
            base_activation=nn.SiLU,
            grid_eps=0.02,
            grid_range=[0, 1]
        )

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.maxpool(x)
        
        x = self.flatten(x)
        x = self.kan1(x)
        
        x = F.log_softmax(x, dim=1)  

        return x
    
model = NormalizedConvolutionalKAN()


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

start_time = time.time()

for epoch in range(10): 
    epoch_start_time = time.time()
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_time = time.time() - epoch_start_time
    print(f'Epoch [{epoch + 1}/10], Loss: {running_loss / len(train_loader):.4f}, Time elapsed: {epoch_time:.2f} seconds')

total_time = time.time() - start_time
print(f"Training completed in: {total_time:.2f} seconds")


model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')

print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')
torch.save(model.state_dict(), 'kan_c_bn_64x64.pth')

Epoch [1/10], Loss: 0.5336, Time elapsed: 89.02 seconds
Epoch [2/10], Loss: 0.0786, Time elapsed: 93.35 seconds
Epoch [3/10], Loss: 0.0303, Time elapsed: 95.18 seconds
Epoch [4/10], Loss: 0.0126, Time elapsed: 88.82 seconds
Epoch [5/10], Loss: 0.0069, Time elapsed: 90.64 seconds
Epoch [6/10], Loss: 0.0038, Time elapsed: 93.18 seconds
Epoch [7/10], Loss: 0.0021, Time elapsed: 95.00 seconds
Epoch [8/10], Loss: 0.0015, Time elapsed: 99.58 seconds
Epoch [9/10], Loss: 0.0011, Time elapsed: 99.93 seconds
Epoch [10/10], Loss: 0.0008, Time elapsed: 103.38 seconds
Training completed in: 948.08 seconds
Accuracy: 0.9956, Precision: 0.9962, Recall: 0.9961, F1 Score: 0.9962


In [3]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import math
import seaborn as sns
import time


from kan_convolutional.KANLinear import KANLinear
from kan_convolutional.KANConv import KAN_Convolutional_Layer
from kan_convolutional import convolution 



import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class Data_Class(Dataset):
    def __init__(self, root_dirs, transform=None):
        self.transform = transform
        self.image_files = []
        self.labels = []
        self.class_names = []

        for root_dir in root_dirs:
            for label, subfolder in enumerate(os.listdir(root_dir)):
                subfolder_path = os.path.join(root_dir, subfolder)
                if os.path.isdir(subfolder_path):
                    if subfolder not in self.class_names:
                        self.class_names.append(subfolder)
                    label = self.class_names.index(subfolder)
                    for img_file in os.listdir(subfolder_path):
                        if img_file.endswith('.png'):
                            self.image_files.append(os.path.join(subfolder_path, img_file))
                            self.labels.append(label)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        image = Image.open(img_name).convert('L')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

    def get_class_names(self):
        return self.class_names

root_dirs = [
    "aug"
]

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = Data_Class(root_dirs=root_dirs, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvolutionalKAN(nn.Module):
    def __init__(self):
        super(ConvolutionalKAN, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 5, kernel_size=3, padding=1)
        
        self.conv2 = nn.Conv2d(5, 25, kernel_size=3, padding=1)

        self.maxpool = nn.MaxPool2d(kernel_size=(2, 2))

        self.flatten = nn.Flatten()

        self.kan1 = KANLinear(
            in_features=25 * 16 * 16,
            out_features=9, # out_features will change as per the number of output class of the dataset being considered. 
            grid_size=10,
            spline_order=3,
            scale_noise=0.01,
            scale_base=1,
            scale_spline=1,
            base_activation=nn.SiLU,
            grid_eps=0.02,
            grid_range=[0, 1]
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        
        x = self.conv2(x)
        x = self.maxpool(x)
        
        x = self.flatten(x)
        x = self.kan1(x)
        
        x = F.log_softmax(x, dim=1)  

        return x
    
model = ConvolutionalKAN()


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

start_time = time.time()

for epoch in range(10): 
    epoch_start_time = time.time()
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_time = time.time() - epoch_start_time
    print(f'Epoch [{epoch + 1}/10], Loss: {running_loss / len(train_loader):.4f}, Time elapsed: {epoch_time:.2f} seconds')

total_time = time.time() - start_time
print(f"Training completed in: {total_time:.2f} seconds")


model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')

print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')
torch.save(model.state_dict(), 'kan_c_64x64.pth')

Epoch [1/10], Loss: 1.1543, Time elapsed: 107.98 seconds
Epoch [2/10], Loss: 0.5174, Time elapsed: 105.14 seconds
Epoch [3/10], Loss: 0.2592, Time elapsed: 109.31 seconds
Epoch [4/10], Loss: 0.1367, Time elapsed: 118.58 seconds
Epoch [5/10], Loss: 0.0867, Time elapsed: 121.83 seconds
Epoch [6/10], Loss: 0.0615, Time elapsed: 126.64 seconds
Epoch [7/10], Loss: 0.0482, Time elapsed: 121.67 seconds
Epoch [8/10], Loss: 0.0379, Time elapsed: 121.63 seconds
Epoch [9/10], Loss: 0.0297, Time elapsed: 122.16 seconds
Epoch [10/10], Loss: 0.0226, Time elapsed: 118.72 seconds
Training completed in: 1173.66 seconds
Accuracy: 0.9932, Precision: 0.9938, Recall: 0.9939, F1 Score: 0.9939


In [4]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import math
import seaborn as sns
import time


from kan_convolutional.KANLinear import KANLinear
from kan_convolutional.KANConv import KAN_Convolutional_Layer
from kan_convolutional import convolution 

import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class Data_Class(Dataset):
    def __init__(self, root_dirs, transform=None):
        self.transform = transform
        self.image_files = []
        self.labels = []
        self.class_names = []

        for root_dir in root_dirs:
            for label, subfolder in enumerate(os.listdir(root_dir)):
                subfolder_path = os.path.join(root_dir, subfolder)
                if os.path.isdir(subfolder_path):
                    if subfolder not in self.class_names:
                        self.class_names.append(subfolder)
                    label = self.class_names.index(subfolder)
                    for img_file in os.listdir(subfolder_path):
                        if img_file.endswith('.png'):
                            self.image_files.append(os.path.join(subfolder_path, img_file))
                            self.labels.append(label)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        image = Image.open(img_name).convert('L')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

    def get_class_names(self):
        return self.class_names

root_dirs = [
    "aug"
]

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = Data_Class(root_dirs=root_dirs, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


class SimpleLinear(nn.Module):
    def __init__(self):
        super(SimpleLinear, self).__init__()
        self.kan = KANLinear(
            in_features=64 * 64,
            out_features=9, # out_features will change as per the number of output class of the dataset being considered. 
            grid_size=10,
            spline_order=3,
            scale_noise=0.01,
            scale_base=1,
            scale_spline=1,
            base_activation=nn.SiLU,
            grid_eps=0.02,
            grid_range=[0, 1]
        )
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.flatten(x)
        x = self.kan(x)
        x = F.log_softmax(x, dim=1)
        return x

model = SimpleLinear()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

start_time = time.time()

for epoch in range(10): 
    epoch_start_time = time.time()
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_time = time.time() - epoch_start_time
    print(f'Epoch [{epoch + 1}/10], Loss: {running_loss / len(train_loader):.4f}, Time elapsed: {epoch_time:.2f} seconds')

total_time = time.time() - start_time
print(f"Training completed in: {total_time:.2f} seconds")


model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')

print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')
torch.save(model.state_dict(), 'kan_l_64x64.pth')

Epoch [1/10], Loss: 1.1672, Time elapsed: 69.14 seconds
Epoch [2/10], Loss: 0.4149, Time elapsed: 60.87 seconds
Epoch [3/10], Loss: 0.2031, Time elapsed: 56.43 seconds
Epoch [4/10], Loss: 0.1262, Time elapsed: 55.37 seconds
Epoch [5/10], Loss: 0.0946, Time elapsed: 52.98 seconds
Epoch [6/10], Loss: 0.0740, Time elapsed: 60.39 seconds
Epoch [7/10], Loss: 0.0629, Time elapsed: 63.96 seconds
Epoch [8/10], Loss: 0.0476, Time elapsed: 64.93 seconds
Epoch [9/10], Loss: 0.0379, Time elapsed: 65.65 seconds
Epoch [10/10], Loss: 0.0324, Time elapsed: 70.40 seconds
Training completed in: 620.13 seconds
Accuracy: 0.9883, Precision: 0.9905, Recall: 0.9886, F1 Score: 0.9893


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
from torch.optim import Adam
import os
import time
from PIL import Image
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import numpy as np

# Dataset class
class Data_Class(Dataset):
    def __init__(self, root_dirs, transform=None):
        self.root_dirs = root_dirs
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(root_dirs[0]))
        
        for root_dir in root_dirs:
            for label, class_name in enumerate(self.classes):
                class_dir = os.path.join(root_dir, class_name)
                for img_name in os.listdir(class_dir):
                    img_path = os.path.join(class_dir, img_name)
                    self.image_paths.append(img_path)
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("L")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

class Inception(nn.Module):
    def __init__(self, in_channels, out_1x1, out_3x3_reduce, out_3x3, out_5x5_reduce, out_5x5, out_pool_proj):
        super(Inception, self).__init__()
        
        self.branch1 = nn.Conv2d(in_channels, out_1x1, kernel_size=1)

        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, out_3x3_reduce, kernel_size=1),
            nn.Conv2d(out_3x3_reduce, out_3x3, kernel_size=3, padding=1)
        )
        
        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, out_5x5_reduce, kernel_size=1),
            nn.Conv2d(out_5x5_reduce, out_5x5, kernel_size=5, padding=2)
        )
        
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, out_pool_proj, kernel_size=1)
        )
    
    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        
        outputs = [branch1, branch2, branch3, branch4]
        return torch.cat(outputs, 1)

class GoogleNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super(GoogleNetClassifier, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.conv2 = nn.Conv2d(64, 192, kernel_size=3, stride=1, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024, num_classes)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        
        x = self.conv2(x)
        x = self.maxpool2(x)
        
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)
        
        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)
        
        x = self.inception5a(x)
        x = self.inception5b(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        
        return x

image_size = 64
num_classes = 9 # num_classes will change as per the number of output class of the dataset being considered. 

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor()
])

root_dirs = ['aug']
dataset = Data_Class(root_dirs=root_dirs, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model = GoogleNetClassifier(num_classes=num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-4)

start_time = time.time()
num_epochs = 10
for epoch in range(num_epochs):
    epoch_start_time = time.time()
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    epoch_time = time.time() - epoch_start_time
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Time elapsed: {epoch_time:.2f} seconds")


total_time = time.time() - start_time
print(f"Training completed in: {total_time:.2f} seconds")


model.eval()

all_labels = []
all_predictions = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)

accuracy = accuracy_score(all_labels, all_predictions)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='macro')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

print('\nClassification Report:')
print(classification_report(all_labels, all_predictions, target_names=[f'Class {i}' for i in range(num_classes)]))

torch.save(model.state_dict(), 'googlenet_64x64.pth')

Epoch [1/10], Loss: 1.8736, Time elapsed: 109.29 seconds
Epoch [2/10], Loss: 0.9522, Time elapsed: 112.31 seconds
Epoch [3/10], Loss: 0.7994, Time elapsed: 121.61 seconds
Epoch [4/10], Loss: 0.7362, Time elapsed: 162.00 seconds
Epoch [5/10], Loss: 0.7099, Time elapsed: 175.27 seconds
Epoch [6/10], Loss: 0.5770, Time elapsed: 157.71 seconds
Epoch [7/10], Loss: 0.4247, Time elapsed: 159.87 seconds
Epoch [8/10], Loss: 0.2759, Time elapsed: 172.21 seconds
Epoch [9/10], Loss: 0.2825, Time elapsed: 188.69 seconds
Epoch [10/10], Loss: 0.2261, Time elapsed: 190.45 seconds
Training completed in: 1549.42 seconds
Accuracy: 0.9106
Precision: 0.9186
Recall: 0.9204
F1 Score: 0.9192

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.71      0.68      0.70       245
     Class 1       0.71      0.71      0.71       270
     Class 2       0.94      0.91      0.93       265
     Class 3       0.91      0.99      0.95       227
     Class 4       1.00     

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
from torch.optim import Adam
import os
import time
from PIL import Image
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import numpy as np

# Dataset class
class Data_Class(Dataset):
    def __init__(self, root_dirs, transform=None):
        self.root_dirs = root_dirs
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(root_dirs[0]))
        
        for root_dir in root_dirs:
            for label, class_name in enumerate(self.classes):
                class_dir = os.path.join(root_dir, class_name)
                for img_name in os.listdir(class_dir):
                    img_path = os.path.join(class_dir, img_name)
                    self.image_paths.append(img_path)
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("L")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

class Patches(nn.Module):
    def __init__(self, patch_size, in_channels=1):
        super(Patches, self).__init__()
        self.patch_size = patch_size
        self.unfold = nn.Unfold(kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        patches = self.unfold(x)
        patches = patches.transpose(1, 2)
        return patches

class PatchEncoder(nn.Module):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.projection = nn.Linear(16*16, projection_dim)
        self.position_embedding = nn.Parameter(torch.randn(1, num_patches, projection_dim))

    def forward(self, patches):
        encoded = self.projection(patches) + self.position_embedding
        return encoded

class MLP(nn.Module):
    def __init__(self, hidden_units, dropout_rate):
        super(MLP, self).__init__()
        layers = []
        for units in hidden_units:
            layers.append(nn.Linear(units[0], units[1]))
            layers.append(nn.GELU())
            layers.append(nn.Dropout(dropout_rate))
        self.mlp = nn.Sequential(*layers)

    def forward(self, x):
        return self.mlp(x)

class VisionTransformer(nn.Module):
    def __init__(self, image_size, patch_size, num_patches, projection_dim, num_heads, transformer_layers, transformer_units, mlp_head_units, num_classes):
        super(VisionTransformer, self).__init__()
        self.patches = Patches(patch_size=patch_size)
        self.patch_encoder = PatchEncoder(num_patches=num_patches, projection_dim=projection_dim)
        self.transformer_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=projection_dim,
                nhead=num_heads,
                dim_feedforward=transformer_units[0][1],
                dropout=0.1
            )
            for _ in range(transformer_layers)
        ])
        self.mlp_head = MLP(mlp_head_units, dropout_rate=0.5)
        self.classifier = nn.Linear(mlp_head_units[-1][1], num_classes)

    def forward(self, x):
        patches = self.patches(x)
        encoded_patches = self.patch_encoder(patches)

        for transformer_layer in self.transformer_layers:
            encoded_patches = transformer_layer(encoded_patches)

        representation = encoded_patches.mean(dim=1)
        features = self.mlp_head(representation)
        logits = self.classifier(features)

        return logits

image_size = 64
patch_size = 16
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 8
transformer_layers = 6
transformer_units = [(projection_dim, projection_dim * 4)]
mlp_head_units = [(projection_dim, projection_dim * 2), (projection_dim * 2, 512)]
num_classes = 9 # num_classes will change as per the number of output class of the dataset being considered. 

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor()
])

root_dirs = ['aug']
dataset = Data_Class(root_dirs=root_dirs, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model = VisionTransformer(image_size=image_size, patch_size=patch_size, num_patches=num_patches,
                          projection_dim=projection_dim, num_heads=num_heads, transformer_layers=transformer_layers,
                          transformer_units=transformer_units, mlp_head_units=mlp_head_units, num_classes=num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-4)

start_time = time.time()
num_epochs = 10
for epoch in range(num_epochs):
    epoch_start_time = time.time()
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    epoch_time = time.time() - epoch_start_time
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}, Time elapsed: {epoch_time:.2f} seconds")


total_time = time.time() - start_time
print(f"Training completed in: {total_time:.2f} seconds")

model.eval()
all_labels = []
all_predictions = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)

accuracy = accuracy_score(all_labels, all_predictions)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='macro')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

print('\nClassification Report:')
print(classification_report(all_labels, all_predictions, target_names=[f'Class {i}' for i in range(num_classes)]))

torch.save(model.state_dict(), 'vit_64x64.pth')

Epoch [1/10], Loss: 2.1957, Time elapsed: 74.31 seconds
Epoch [2/10], Loss: 1.9978, Time elapsed: 71.27 seconds
Epoch [3/10], Loss: 1.4949, Time elapsed: 63.06 seconds
Epoch [4/10], Loss: 1.3599, Time elapsed: 63.60 seconds
Epoch [5/10], Loss: 1.2490, Time elapsed: 65.27 seconds
Epoch [6/10], Loss: 1.1682, Time elapsed: 66.90 seconds
Epoch [7/10], Loss: 1.1078, Time elapsed: 70.12 seconds
Epoch [8/10], Loss: 1.0446, Time elapsed: 71.53 seconds
Epoch [9/10], Loss: 0.9991, Time elapsed: 63.08 seconds
Epoch [10/10], Loss: 0.9302, Time elapsed: 61.22 seconds
Training completed in: 670.37 seconds
Accuracy: 0.6302
Precision: 0.5875
Recall: 0.6452
F1 Score: 0.5825

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.00      0.00      0.00       243
     Class 1       0.27      1.00      0.43       269
     Class 2       0.00      0.00      0.00       233
     Class 3       0.20      0.00      0.01       245
     Class 4       1.00      0.93      

C:\Users\Anurag Dutta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Anurag Dutta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Anurag Dutta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metric

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
from torch.optim import Adam
import os
import time
from PIL import Image
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import numpy as np

# Dataset class
class Data_Class(Dataset):
    def __init__(self, root_dirs, transform=None):
        self.root_dirs = root_dirs
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(root_dirs[0]))
        
        for root_dir in root_dirs:
            for label, class_name in enumerate(self.classes):
                class_dir = os.path.join(root_dir, class_name)
                for img_name in os.listdir(class_dir):
                    img_path = os.path.join(class_dir, img_name)
                    self.image_paths.append(img_path)
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("L")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity
        out = self.relu(out)

        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

def resnet18(num_classes):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)

image_size = 64
num_classes = 9 # num_classes will change as per the number of output class of the dataset being considered. 

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor()
])

root_dirs = ['aug']
dataset = Data_Class(root_dirs=root_dirs, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model = resnet18(num_classes=num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-4)

start_time = time.time()
num_epochs = 10
for epoch in range(num_epochs):
    epoch_start_time = time.time()
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    epoch_time = time.time() - epoch_start_time
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Time elapsed: {epoch_time:.2f} seconds")


total_time = time.time() - start_time
print(f"Training completed in: {total_time:.2f} seconds")

model.eval()
all_labels = []
all_predictions = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)

accuracy = accuracy_score(all_labels, all_predictions)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='macro')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

print('\nClassification Report:')
print(classification_report(all_labels, all_predictions, target_names=[f'Class {i}' for i in range(num_classes)]))

torch.save(model.state_dict(), 'resnet_64x64.pth')

Epoch [1/10], Loss: 0.3036, Time elapsed: 256.80 seconds
Epoch [2/10], Loss: 0.0869, Time elapsed: 293.45 seconds
Epoch [3/10], Loss: 0.0493, Time elapsed: 297.83 seconds
Epoch [4/10], Loss: 0.0306, Time elapsed: 293.64 seconds
Epoch [5/10], Loss: 0.0270, Time elapsed: 227.83 seconds
Epoch [6/10], Loss: 0.0313, Time elapsed: 159.57 seconds
Epoch [7/10], Loss: 0.0192, Time elapsed: 140.64 seconds
Epoch [8/10], Loss: 0.0120, Time elapsed: 157.60 seconds
Epoch [9/10], Loss: 0.0317, Time elapsed: 159.82 seconds
Epoch [10/10], Loss: 0.0126, Time elapsed: 170.02 seconds
Training completed in: 2157.21 seconds
Accuracy: 0.9947
Precision: 0.9951
Recall: 0.9950
F1 Score: 0.9950

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.99      0.97      0.98       239
     Class 1       0.98      0.99      0.98       247
     Class 2       0.99      1.00      0.99       280
     Class 3       1.00      1.00      1.00       228
     Class 4       1.00     

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
from torch.optim import Adam
import os
import time
from PIL import Image
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import numpy as np

# Dataset class
class Data_Class(Dataset):
    def __init__(self, root_dirs, transform=None):
        self.root_dirs = root_dirs
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(root_dirs[0]))
        
        for root_dir in root_dirs:
            for label, class_name in enumerate(self.classes):
                class_dir = os.path.join(root_dir, class_name)
                for img_name in os.listdir(class_dir):
                    img_path = os.path.join(class_dir, img_name)
                    self.image_paths.append(img_path)
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("L")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


class CNNClassifier(nn.Module):
    def __init__(self, num_classes):
        super(CNNClassifier, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 5, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(5, 25, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(25 * 16 * 16, 512),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

image_size = 64
num_classes = 9 # num_classes will change as per the number of output class of the dataset being considered. 

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor()
])

root_dirs = ['aug']
dataset = Data_Class(root_dirs=root_dirs, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model = CNNClassifier(num_classes=num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-4)

start_time = time.time()
num_epochs = 10
for epoch in range(num_epochs):
    epoch_start_time = time.time()
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    epoch_time = time.time() - epoch_start_time
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Time elapsed: {epoch_time:.2f} seconds")


total_time = time.time() - start_time
print(f"Training completed in: {total_time:.2f} seconds")

model.eval()
all_labels = []
all_predictions = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)

accuracy = accuracy_score(all_labels, all_predictions)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='macro')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

print('\nClassification Report:')
print(classification_report(all_labels, all_predictions, target_names=[f'Class {i}' for i in range(num_classes)]))

torch.save(model.state_dict(), 'cnn_64x64.pth')

Epoch [1/10], Loss: 2.1876, Time elapsed: 96.87 seconds
Epoch [2/10], Loss: 1.8415, Time elapsed: 70.81 seconds
Epoch [3/10], Loss: 1.5181, Time elapsed: 59.30 seconds
Epoch [4/10], Loss: 1.3740, Time elapsed: 63.37 seconds
Epoch [5/10], Loss: 1.2127, Time elapsed: 60.30 seconds
Epoch [6/10], Loss: 1.0720, Time elapsed: 70.48 seconds
Epoch [7/10], Loss: 0.9735, Time elapsed: 64.88 seconds
Epoch [8/10], Loss: 0.9131, Time elapsed: 64.51 seconds
Epoch [9/10], Loss: 0.8736, Time elapsed: 67.73 seconds
Epoch [10/10], Loss: 0.8400, Time elapsed: 58.53 seconds
Training completed in: 676.78 seconds
Accuracy: 0.6283
Precision: 0.6144
Recall: 0.6611
F1 Score: 0.5950

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.00      0.00      0.00       229
     Class 1       0.00      0.00      0.00       277
     Class 2       0.25      1.00      0.40       250
     Class 3       0.33      0.00      0.01       248
     Class 4       1.00      0.97      

C:\Users\Anurag Dutta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Anurag Dutta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Anurag Dutta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metric

In [2]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
import os
import time
from PIL import Image
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np

class Data_Class(Dataset):
    def __init__(self, root_dirs, transform=None):
        self.root_dirs = root_dirs
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(root_dirs[0]))
        
        for root_dir in root_dirs:
            for label, class_name in enumerate(self.classes):
                class_dir = os.path.join(root_dir, class_name)
                for img_name in os.listdir(class_dir):
                    img_path = os.path.join(class_dir, img_name)
                    self.image_paths.append(img_path)
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("L")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

image_size = 64
num_classes = 9 # num_classes will change as per the number of output class of the dataset being considered. 

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor()
])

root_dirs = ['aug']
dataset = Data_Class(root_dirs=root_dirs, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

def flatten_images(loader):
    flattened_images = []
    labels = []
    for images, lbls in loader:
        images = images.view(images.size(0), -1).numpy()
        flattened_images.append(images)
        labels.append(lbls.numpy())
    return np.vstack(flattened_images), np.hstack(labels)

X_train, y_train = flatten_images(train_loader)
X_test, y_test = flatten_images(test_loader)

model = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(kernel='rbf', C=1, gamma='scale', probability=True))
])

start_time = time.time()
model.fit(X_train, y_train)
training_time = time.time() - start_time

print(f"Training completed in: {training_time:.2f} seconds")

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

print('\nClassification Report:')
print(classification_report(y_test, y_pred, target_names=[f'Class {i}' for i in range(num_classes)]))

import joblib
joblib.dump(model, 'svm_64x64.pkl')

Training completed in: 322.80 seconds
Accuracy: 0.9845
Precision: 0.9875
Recall: 0.9850
F1 Score: 0.9859

Classification Report:
              precision    recall  f1-score   support

     Class 0       1.00      0.94      0.97       221
     Class 1       0.91      1.00      0.95       279
     Class 2       1.00      0.96      0.98       263
     Class 3       1.00      0.99      0.99       258
     Class 4       1.00      1.00      1.00       221
     Class 5       0.99      1.00      1.00       210
     Class 6       1.00      1.00      1.00       202
     Class 7       1.00      1.00      1.00       213
     Class 8       0.99      0.99      0.99       191

    accuracy                           0.98      2058
   macro avg       0.99      0.98      0.99      2058
weighted avg       0.99      0.98      0.98      2058



['svm_64x64.pkl']

In [3]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
import os
import time
from PIL import Image
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
import joblib
from xgboost import XGBClassifier

class Data_Class(Dataset):
    def __init__(self, root_dirs, transform=None):
        self.root_dirs = root_dirs
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(root_dirs[0]))
        
        for root_dir in root_dirs:
            for label, class_name in enumerate(self.classes):
                class_dir = os.path.join(root_dir, class_name)
                for img_name in os.listdir(class_dir):
                    img_path = os.path.join(class_dir, img_name)
                    self.image_paths.append(img_path)
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("L")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

image_size = 64
num_classes = 9 # num_classes will change as per the number of output class of the dataset being considered. 

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor()
])

root_dirs = ['aug']
dataset = Data_Class(root_dirs=root_dirs, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

def flatten_images(loader):
    flattened_images = []
    labels = []
    for images, lbls in loader:
        images = images.view(images.size(0), -1).numpy()
        flattened_images.append(images)
        labels.append(lbls.numpy())
    return np.vstack(flattened_images), np.hstack(labels)

X_train, y_train = flatten_images(train_loader)
X_test, y_test = flatten_images(test_loader)

model = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', n_estimators=100, max_depth=6, learning_rate=0.1))
])

start_time = time.time()
model.fit(X_train, y_train)
training_time = time.time() - start_time

print(f"Training completed in: {training_time:.2f} seconds")

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

print('\nClassification Report:')
print(classification_report(y_test, y_pred, target_names=[f'Class {i}' for i in range(num_classes)]))

joblib.dump(model, 'xgboost_64x64.pkl')

C:\Users\Anurag Dutta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


Training completed in: 737.37 seconds
Accuracy: 0.9845
Precision: 0.9855
Recall: 0.9858
F1 Score: 0.9856

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.96      0.96      0.96       246
     Class 1       0.97      0.96      0.96       271
     Class 2       0.96      0.98      0.97       230
     Class 3       1.00      0.98      0.99       264
     Class 4       1.00      1.00      1.00       217
     Class 5       1.00      1.00      1.00       206
     Class 6       1.00      1.00      1.00       211
     Class 7       1.00      1.00      1.00       201
     Class 8       1.00      1.00      1.00       212

    accuracy                           0.98      2058
   macro avg       0.99      0.99      0.99      2058
weighted avg       0.98      0.98      0.98      2058



['xgboost_64x64.pkl']

In [4]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
import os
import time
from PIL import Image
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
import joblib

class Data_Class(Dataset):
    def __init__(self, root_dirs, transform=None):
        self.root_dirs = root_dirs
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(root_dirs[0]))
        
        for root_dir in root_dirs:
            for label, class_name in enumerate(self.classes):
                class_dir = os.path.join(root_dir, class_name)
                for img_name in os.listdir(class_dir):
                    img_path = os.path.join(class_dir, img_name)
                    self.image_paths.append(img_path)
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("L")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

image_size = 64
num_classes = 9 # num_classes will change as per the number of output class of the dataset being considered. 

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor()
])

root_dirs = ['aug']
dataset = Data_Class(root_dirs=root_dirs, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

def flatten_images(loader):
    flattened_images = []
    labels = []
    for images, lbls in loader:
        images = images.view(images.size(0), -1).numpy()
        flattened_images.append(images)
        labels.append(lbls.numpy())
    return np.vstack(flattened_images), np.hstack(labels)

X_train, y_train = flatten_images(train_loader)
X_test, y_test = flatten_images(test_loader)

model = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42))
])

start_time = time.time()
model.fit(X_train, y_train)
training_time = time.time() - start_time

print(f"Training completed in: {training_time:.2f} seconds")

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

print('\nClassification Report:')
print(classification_report(y_test, y_pred, target_names=[f'Class {i}' for i in range(num_classes)]))

joblib.dump(model, 'rf_64x64.pkl')

Training completed in: 18.06 seconds
Accuracy: 0.9738
Precision: 0.9765
Recall: 0.9761
F1 Score: 0.9762

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.93      0.92      0.93       246
     Class 1       0.91      0.94      0.93       269
     Class 2       0.98      0.95      0.96       256
     Class 3       0.99      0.99      0.99       232
     Class 4       1.00      0.99      0.99       231
     Class 5       0.99      1.00      1.00       223
     Class 6       1.00      1.00      1.00       207
     Class 7       1.00      1.00      1.00       209
     Class 8       0.99      0.99      0.99       185

    accuracy                           0.97      2058
   macro avg       0.98      0.98      0.98      2058
weighted avg       0.97      0.97      0.97      2058



['rf_64x64.pkl']

In [5]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
import os
import time
from PIL import Image
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
import joblib

class Data_Class(Dataset):
    def __init__(self, root_dirs, transform=None):
        self.root_dirs = root_dirs
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(root_dirs[0]))
        
        for root_dir in root_dirs:
            for label, class_name in enumerate(self.classes):
                class_dir = os.path.join(root_dir, class_name)
                for img_name in os.listdir(class_dir):
                    img_path = os.path.join(class_dir, img_name)
                    self.image_paths.append(img_path)
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("L")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

image_size = 64
num_classes = 9 # num_classes will change as per the number of output class of the dataset being considered. 

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor()
])

root_dirs = ['aug']
dataset = Data_Class(root_dirs=root_dirs, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

def flatten_images(loader):
    flattened_images = []
    labels = []
    for images, lbls in loader:
        images = images.view(images.size(0), -1).numpy()
        flattened_images.append(images)
        labels.append(lbls.numpy())
    return np.vstack(flattened_images), np.hstack(labels)

X_train, y_train = flatten_images(train_loader)
X_test, y_test = flatten_images(test_loader)

model = Pipeline([
    ('scaler', StandardScaler()),
    ('logistic', LogisticRegression(max_iter=1000, multi_class='ovr'))
])

start_time = time.time()
model.fit(X_train, y_train)
training_time = time.time() - start_time

print(f"Training completed in: {training_time:.2f} seconds")

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

print('\nClassification Report:')
print(classification_report(y_test, y_pred, target_names=[f'Class {i}' for i in range(num_classes)]))

joblib.dump(model, 'logistic_64x64.pkl')

C:\Users\Anurag Dutta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


Training completed in: 18.63 seconds
Accuracy: 0.9971
Precision: 0.9975
Recall: 0.9973
F1 Score: 0.9974

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.99      0.99      0.99       249
     Class 1       0.99      0.99      0.99       267
     Class 2       1.00      1.00      1.00       267
     Class 3       1.00      1.00      1.00       235
     Class 4       1.00      1.00      1.00       230
     Class 5       1.00      1.00      1.00       205
     Class 6       1.00      1.00      1.00       188
     Class 7       1.00      1.00      1.00       214
     Class 8       1.00      1.00      1.00       203

    accuracy                           1.00      2058
   macro avg       1.00      1.00      1.00      2058
weighted avg       1.00      1.00      1.00      2058



['logistic_64x64.pkl']

In [6]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
import os
import time
from PIL import Image
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
import joblib

class Data_Class(Dataset):
    def __init__(self, root_dirs, transform=None):
        self.root_dirs = root_dirs
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(root_dirs[0]))
        
        for root_dir in root_dirs:
            for label, class_name in enumerate(self.classes):
                class_dir = os.path.join(root_dir, class_name)
                for img_name in os.listdir(class_dir):
                    img_path = os.path.join(class_dir, img_name)
                    self.image_paths.append(img_path)
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("L")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

image_size = 64
num_classes = 9 # num_classes will change as per the number of output class of the dataset being considered. 

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor()
])

root_dirs = ['aug']
dataset = Data_Class(root_dirs=root_dirs, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

def flatten_images(loader):
    flattened_images = []
    labels = []
    for images, lbls in loader:
        images = images.view(images.size(0), -1).numpy()
        flattened_images.append(images)
        labels.append(lbls.numpy())
    return np.vstack(flattened_images), np.hstack(labels)

X_train, y_train = flatten_images(train_loader)
X_test, y_test = flatten_images(test_loader)

model = Pipeline([
    ('scaler', StandardScaler()),
    ('nb', GaussianNB())
])

start_time = time.time()
model.fit(X_train, y_train)
training_time = time.time() - start_time

print(f"Training completed in: {training_time:.2f} seconds")

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

print('\nClassification Report:')
print(classification_report(y_test, y_pred, target_names=[f'Class {i}' for i in range(num_classes)]))

joblib.dump(model, 'naive_bayes_64x64.pkl')

Training completed in: 0.80 seconds
Accuracy: 0.7478
Precision: 0.8027
Recall: 0.7719
F1 Score: 0.7537

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.63      0.85      0.72       242
     Class 1       1.00      0.31      0.47       257
     Class 2       0.41      0.21      0.28       270
     Class 3       0.48      0.89      0.62       251
     Class 4       0.97      0.90      0.93       239
     Class 5       0.82      1.00      0.90       220
     Class 6       1.00      0.96      0.98       206
     Class 7       1.00      0.96      0.98       190
     Class 8       0.92      0.88      0.90       183

    accuracy                           0.75      2058
   macro avg       0.80      0.77      0.75      2058
weighted avg       0.79      0.75      0.73      2058



['naive_bayes_64x64.pkl']

In [7]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
import os
import time
from PIL import Image
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
import joblib

class Data_Class(Dataset):
    def __init__(self, root_dirs, transform=None):
        self.root_dirs = root_dirs
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(root_dirs[0]))
        
        for root_dir in root_dirs:
            for label, class_name in enumerate(self.classes):
                class_dir = os.path.join(root_dir, class_name)
                for img_name in os.listdir(class_dir):
                    img_path = os.path.join(class_dir, img_name)
                    self.image_paths.append(img_path)
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("L")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

image_size = 64
num_classes = 9 # num_classes will change as per the number of output class of the dataset being considered. 

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor()
])

root_dirs = ['aug']
dataset = Data_Class(root_dirs=root_dirs, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

def flatten_images(loader):
    flattened_images = []
    labels = []
    for images, lbls in loader:
        images = images.view(images.size(0), -1).numpy()
        flattened_images.append(images)
        labels.append(lbls.numpy())
    return np.vstack(flattened_images), np.hstack(labels)

X_train, y_train = flatten_images(train_loader)
X_test, y_test = flatten_images(test_loader)

model = Pipeline([
    ('scaler', StandardScaler()),
    ('dt', DecisionTreeClassifier())
])

start_time = time.time()
model.fit(X_train, y_train)
training_time = time.time() - start_time

print(f"Training completed in: {training_time:.2f} seconds")

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

print('\nClassification Report:')
print(classification_report(y_test, y_pred, target_names=[f'Class {i}' for i in range(num_classes)]))

joblib.dump(model, 'dt_64x64.pkl')

Training completed in: 16.09 seconds
Accuracy: 0.8571
Precision: 0.8700
Recall: 0.8680
F1 Score: 0.8685

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.67      0.67      0.67       233
     Class 1       0.70      0.71      0.71       286
     Class 2       0.67      0.74      0.71       233
     Class 3       0.89      0.80      0.84       258
     Class 4       0.97      0.99      0.98       223
     Class 5       0.98      0.96      0.97       204
     Class 6       1.00      1.00      1.00       211
     Class 7       0.99      1.00      0.99       210
     Class 8       0.96      0.95      0.95       200

    accuracy                           0.86      2058
   macro avg       0.87      0.87      0.87      2058
weighted avg       0.86      0.86      0.86      2058



['dt_64x64.pkl']

In [1]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import math
import seaborn as sns
import time


from kan_convolutional.KANLinear import KANLinear
from kan_convolutional.KANConv import KAN_Convolutional_Layer
from kan_convolutional import convolution 

import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class Data_Class(Dataset):
    def __init__(self, root_dirs, transform=None):
        self.transform = transform
        self.image_files = []
        self.labels = []
        self.class_names = []

        for root_dir in root_dirs:
            for label, subfolder in enumerate(os.listdir(root_dir)):
                subfolder_path = os.path.join(root_dir, subfolder)
                if os.path.isdir(subfolder_path):
                    if subfolder not in self.class_names:
                        self.class_names.append(subfolder)
                    label = self.class_names.index(subfolder)
                    for img_file in os.listdir(subfolder_path):
                        if img_file.endswith('.png'):
                            self.image_files.append(os.path.join(subfolder_path, img_file))
                            self.labels.append(label)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        image = Image.open(img_name).convert('L')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

    def get_class_names(self):
        return self.class_names

root_dirs = [
    "aug"
]

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = Data_Class(root_dirs=root_dirs, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicResNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.skip_connection = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.skip_connection = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = self.skip_connection(x)
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += residual
        x = F.relu(x)
        return x


class SuperResidualKAN(nn.Module):
    def __init__(self):
        super(SuperResidualKAN, self).__init__()

        # Replace Conv2d layers with ResNet blocks
        self.resnet_block1 = BasicResNetBlock(1, 5)
        self.resnet_block2 = BasicResNetBlock(5, 25)

        self.maxpool = nn.MaxPool2d(kernel_size=(2, 2))

        self.flatten = nn.Flatten()

        self.kan1 = KANLinear(
            in_features=6400,
            out_features=256,
            grid_size=10,
            spline_order=3,
            scale_noise=0.01,
            scale_base=1,
            scale_spline=1,
            base_activation=nn.SiLU,
            grid_eps=0.02,
            grid_range=[0, 1]
        )

        self.kan2 = KANLinear(
            in_features=256,
            out_features=9,  # Adjust for number of classes
            grid_size=10,
            spline_order=3,
            scale_noise=0.01,
            scale_base=1,
            scale_spline=1,
            base_activation=nn.SiLU,
            grid_eps=0.02,
            grid_range=[0, 1]
        )

    def forward(self, x):
        x = self.resnet_block1(x)
        x = self.maxpool(x)

        x = self.resnet_block2(x)
        x = self.maxpool(x)

        x = self.flatten(x)

        x = self.kan1(x)

        x = self.kan2(x)

        x = F.log_softmax(x, dim=1)

        return x

    
model = SuperResidualKAN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

start_time = time.time()

for epoch in range(10): 
    epoch_start_time = time.time()
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_time = time.time() - epoch_start_time
    print(f'Epoch [{epoch + 1}/10], Loss: {running_loss / len(train_loader):.4f}, Time elapsed: {epoch_time:.2f} seconds')

total_time = time.time() - start_time
print(f"Training completed in: {total_time:.2f} seconds")

model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')

print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')
torch.save(model.state_dict(), 'super_kan_r_64x64.pth')

Epoch [1/10], Loss: 0.3876, Time elapsed: 166.93 seconds
Epoch [2/10], Loss: 0.0443, Time elapsed: 215.32 seconds
Epoch [3/10], Loss: 0.0159, Time elapsed: 234.99 seconds
Epoch [4/10], Loss: 0.0077, Time elapsed: 293.57 seconds
Epoch [5/10], Loss: 0.0059, Time elapsed: 251.94 seconds
Epoch [6/10], Loss: 0.0068, Time elapsed: 221.33 seconds
Epoch [7/10], Loss: 0.0083, Time elapsed: 272.43 seconds
Epoch [8/10], Loss: 0.0016, Time elapsed: 279.28 seconds
Epoch [9/10], Loss: 0.0002, Time elapsed: 265.09 seconds
Epoch [10/10], Loss: 0.0001, Time elapsed: 237.29 seconds
Training completed in: 2438.17 seconds
Accuracy: 0.9976, Precision: 0.9978, Recall: 0.9978, F1 Score: 0.9978


In [2]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import math
import seaborn as sns
import time


from kan_convolutional.KANLinear import KANLinear
from kan_convolutional.KANConv import KAN_Convolutional_Layer
from kan_convolutional import convolution 

import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class Data_Class(Dataset):
    def __init__(self, root_dirs, transform=None):
        self.transform = transform
        self.image_files = []
        self.labels = []
        self.class_names = []

        for root_dir in root_dirs:
            for label, subfolder in enumerate(os.listdir(root_dir)):
                subfolder_path = os.path.join(root_dir, subfolder)
                if os.path.isdir(subfolder_path):
                    if subfolder not in self.class_names:
                        self.class_names.append(subfolder)
                    label = self.class_names.index(subfolder)
                    for img_file in os.listdir(subfolder_path):
                        if img_file.endswith('.png'):
                            self.image_files.append(os.path.join(subfolder_path, img_file))
                            self.labels.append(label)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        image = Image.open(img_name).convert('L')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

    def get_class_names(self):
        return self.class_names

root_dirs = [
    "aug"
]

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = Data_Class(root_dirs=root_dirs, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicResNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.skip_connection = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.skip_connection = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = self.skip_connection(x)
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += residual
        x = F.relu(x)
        return x


class NormalizedResidualKAN(nn.Module):
    def __init__(self):
        super(NormalizedResidualKAN, self).__init__()
        
        self.resnet_block1 = BasicResNetBlock(1, 5)
        self.bn1 = nn.BatchNorm2d(5)
        
        self.resnet_block2 = BasicResNetBlock(5, 25)
        self.bn2 = nn.BatchNorm2d(25)

        self.maxpool = nn.MaxPool2d(kernel_size=(2, 2))

        self.flatten = nn.Flatten()

        self.kan1 = KANLinear(
            in_features=25 * 16 * 16,
            out_features=9, # out_features will change as per the number of output class of the dataset being considered. 
            grid_size=10,
            spline_order=3,
            scale_noise=0.01,
            scale_base=1,
            scale_spline=1,
            base_activation=nn.SiLU,
            grid_eps=0.02,
            grid_range=[0, 1]
        )

    def forward(self, x):
        x = F.relu(self.bn1(self.resnet_block1(x)))
        x = self.maxpool(x)
        
        x = F.relu(self.bn2(self.resnet_block2(x)))
        x = self.maxpool(x)
        
        x = self.flatten(x)
        x = self.kan1(x)
        
        x = F.log_softmax(x, dim=1)  

        return x
    
model = NormalizedResidualKAN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

start_time = time.time()

for epoch in range(10): 
    epoch_start_time = time.time()
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_time = time.time() - epoch_start_time
    print(f'Epoch [{epoch + 1}/10], Loss: {running_loss / len(train_loader):.4f}, Time elapsed: {epoch_time:.2f} seconds')

total_time = time.time() - start_time
print(f"Training completed in: {total_time:.2f} seconds")

model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')

print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')
torch.save(model.state_dict(), 'kan_r_bn_64x64.pth')

Epoch [1/10], Loss: 0.3621, Time elapsed: 107.67 seconds
Epoch [2/10], Loss: 0.0584, Time elapsed: 106.89 seconds
Epoch [3/10], Loss: 0.0243, Time elapsed: 128.32 seconds
Epoch [4/10], Loss: 0.0104, Time elapsed: 126.98 seconds
Epoch [5/10], Loss: 0.0169, Time elapsed: 114.97 seconds
Epoch [6/10], Loss: 0.0055, Time elapsed: 120.25 seconds
Epoch [7/10], Loss: 0.0013, Time elapsed: 123.09 seconds
Epoch [8/10], Loss: 0.0005, Time elapsed: 127.54 seconds
Epoch [9/10], Loss: 0.0003, Time elapsed: 134.35 seconds
Epoch [10/10], Loss: 0.0003, Time elapsed: 142.21 seconds
Training completed in: 1232.27 seconds
Accuracy: 0.9917, Precision: 0.9924, Recall: 0.9925, F1 Score: 0.9925


In [3]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import math
import seaborn as sns
import time


from kan_convolutional.KANLinear import KANLinear
from kan_convolutional.KANConv import KAN_Convolutional_Layer
from kan_convolutional import convolution 



import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class Data_Class(Dataset):
    def __init__(self, root_dirs, transform=None):
        self.transform = transform
        self.image_files = []
        self.labels = []
        self.class_names = []

        for root_dir in root_dirs:
            for label, subfolder in enumerate(os.listdir(root_dir)):
                subfolder_path = os.path.join(root_dir, subfolder)
                if os.path.isdir(subfolder_path):
                    if subfolder not in self.class_names:
                        self.class_names.append(subfolder)
                    label = self.class_names.index(subfolder)
                    for img_file in os.listdir(subfolder_path):
                        if img_file.endswith('.png'):
                            self.image_files.append(os.path.join(subfolder_path, img_file))
                            self.labels.append(label)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        image = Image.open(img_name).convert('L')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

    def get_class_names(self):
        return self.class_names

root_dirs = [
    "aug"
]

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = Data_Class(root_dirs=root_dirs, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicResNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.skip_connection = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.skip_connection = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = self.skip_connection(x)
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += residual
        x = F.relu(x)
        return x
    
    
class ResidualKAN(nn.Module):
    def __init__(self):
        super(ResidualKAN, self).__init__()
        
        self.resnet_block1 = BasicResNetBlock(1, 5)
        
        self.resnet_block2 = BasicResNetBlock(5, 25)

        self.maxpool = nn.MaxPool2d(kernel_size=(2, 2))

        self.flatten = nn.Flatten()

        self.kan1 = KANLinear(
            in_features=25 * 16 * 16,
            out_features=9, # out_features will change as per the number of output class of the dataset being considered. 
            grid_size=10,
            spline_order=3,
            scale_noise=0.01,
            scale_base=1,
            scale_spline=1,
            base_activation=nn.SiLU,
            grid_eps=0.02,
            grid_range=[0, 1]
        )

    def forward(self, x):
        x = self.resnet_block1(x)
        x = self.maxpool(x)
        
        x = self.resnet_block2(x)
        x = self.maxpool(x)
        
        x = self.flatten(x)
        x = self.kan1(x)
        
        x = F.log_softmax(x, dim=1)  

        return x
    
model = ResidualKAN()


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

start_time = time.time()

for epoch in range(10): 
    epoch_start_time = time.time()
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_time = time.time() - epoch_start_time
    print(f'Epoch [{epoch + 1}/10], Loss: {running_loss / len(train_loader):.4f}, Time elapsed: {epoch_time:.2f} seconds')

total_time = time.time() - start_time
print(f"Training completed in: {total_time:.2f} seconds")


model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')

print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')
torch.save(model.state_dict(), 'kan_r_64x64.pth')

Epoch [1/10], Loss: 0.4539, Time elapsed: 151.24 seconds
Epoch [2/10], Loss: 0.0471, Time elapsed: 152.68 seconds
Epoch [3/10], Loss: 0.0172, Time elapsed: 114.32 seconds
Epoch [4/10], Loss: 0.0188, Time elapsed: 105.62 seconds
Epoch [5/10], Loss: 0.0083, Time elapsed: 104.32 seconds
Epoch [6/10], Loss: 0.0038, Time elapsed: 106.83 seconds
Epoch [7/10], Loss: 0.0025, Time elapsed: 98.95 seconds
Epoch [8/10], Loss: 0.0005, Time elapsed: 99.93 seconds
Epoch [9/10], Loss: 0.0004, Time elapsed: 101.76 seconds
Epoch [10/10], Loss: 0.0003, Time elapsed: 99.16 seconds
Training completed in: 1134.82 seconds
Accuracy: 0.9966, Precision: 0.9969, Recall: 0.9967, F1 Score: 0.9968
